## Trying out the sketch library

https://colab.research.google.com/gist/bluecoconut/410a979d94613ea2aaf29987cf0233bc/sketch-demo.ipynb

In [ ]:
import pandas as pd
import matplotlib

In [ ]:
POLLUTION_DATA_FILE = "../data/EnterococciDataDownload-20230403-044231.parquet"

In [ ]:
DAILY_BEACH_DATA = "../../../databooth/beach-swim-dataflow/data/daily_beach_data.csv"

In [ ]:
beach_df = pd.read_csv(DAILY_BEACH_DATA)

In [ ]:
beach_df.info()

In [ ]:
df = pd.read_csv(POLLUTION_DATA_FILE.replace(".parquet", ".csv"), parse_dates=True)

df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
# df = pd.read_parquet(POLLUTION_DATA_FILE, )

In [ ]:
df.head()

In [ ]:
merge_df = df.merge(right=beach_df, left_on="Site", right_on="Beach name", how="right", suffixes=["", "_right"])

In [ ]:
merge_df.nunique()

In [ ]:
merge_df.drop(["BeachId", "Region_right", "Data last updated", "Swell", "Wind"], inplace=True, axis=1)

In [ ]:
import sketch

### Try Sketch

In [ ]:
merge_df.sketch.ask("What columns might contain PII data?")

In [ ]:
merge_df.sketch.ask("Can you suggest user-friendly names for each column? (output in order as an Python list)")

In [ ]:
merge_df.columns.to_list()

In [ ]:
df.sketch.howto("Get the place names from the unique latitude and longitude")

In [ ]:
import geopy
from geopy.geocoders import Nominatim

In [ ]:
# Get the place names from the latitude and longitude

# Create a geolocator object
geolocator = Nominatim(user_agent="try-sketch")

# Create an empty list to store the place names
place_names = []

# Iterate through the dataframe rows
counter = 0
for index, row in df.iterrows():
    counter += 1
    # Get the latitude and longitude from the row
    lat = row['Latitude']
    lon = row['Longitude']

    # Get the place name from the geolocator object using the coordinates
    location = geolocator.reverse(f"{lat}, {lon}")

    # Append the place name to the list
    place_names.append(location.address)

    if counter > 10:
        break
    
# Add the list of place names to the dataframe as a new column 
# df['Place Name'] = place_names


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
geolocator = Nominatim(user_agent="try-sketch", timeout=2)

# Get the unique latitude and longitude values
unique_lat_long = merge_df[['Latitude', 'Longitude']].drop_duplicates()

# Get the place names from the unique latitude and longitude
place_names = []
counter = 0

for index, row in unique_lat_long.iterrows():
    counter += 1
    lat = row['Latitude']
    lon = row['Longitude']
    location =  geolocator.reverse(f"{lat}, {lon}")
    place_names.append((location.address, [lat, lon]))

    if counter > 10:
        break
    
# Print the place names
print(place_names)

In [ ]:
place_names

In [ ]:
merge_df.sketch.howto("For each Site plot by the median amount of Enterococci through time removing outliers")

In [ ]:

# import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# set figure size
plt.figure(figsize=(20,10))

# remove outliers from the dataframe
merge_df_no_outliers = merge_df[merge_df['Enterococci (cfu/100ml)'] < merge_df['Enterococci (cfu/100ml)'].quantile(0.95)]

# group by Site and Date and calculate median Enterococci (cfu/100ml)
median_enterococci = merge_df_no_outliers.groupby(['Site','Date'])['Enterococci (cfu/100ml)'].max().reset_index()

# plot the data using seaborn lineplot
sns.lineplot(x='Date', y='Enterococci (cfu/100ml)', hue='Site', data=median_enterococci)
plt.show()


In [ ]:
# import libraries
import matplotlib.pyplot as plt
import seaborn as sns

# set figure size
plt.figure(figsize=(15,8))

# plot median Enterococci by Site over time
sns.lineplot(x="Date", y="Enterococci (cfu/100ml)", hue="Site", data=merge_df, estimator="median")

# show plot
plt.show()


In [ ]:
merge_df.sketch.howto("Show the overall trend of Enterococci through time with a scatter plot with outliers removed")

In [ ]:
import plotly.express as px

In [ ]:
data_df = merge_df[(merge_df['Enterococci (cfu/100ml)'].quantile(0.05) < merge_df['Enterococci (cfu/100ml)']) & 
                   (merge_df['Enterococci (cfu/100ml)'] < merge_df['Enterococci (cfu/100ml)'].quantile(0.95))]

In [ ]:
grouped_df = data_df.groupby("Date")

In [ ]:
grouped_df.head()

In [ ]:
grouped_df.sketch.howto("Plot Enterococci versus date")

In [ ]:
fig = px.scatter(grouped_df, x="Date", y="Enterococci (cfu/100ml)", color="Site")
fig.show()


In [ ]:
import altair as alt


In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
#yscale = alt.Scale(domain=(0, 10000))


alt.Chart(merge_df).mark_circle().encode(
    x='Date:T',
    y=alt.Y('Enterococci (cfu/100ml):Q') #, scale=yscale)
).properties(
    width=600,
    height=400
)


In [ ]:

import matplotlib.pyplot as plt

# Create a scatter plot of Enterococci over time
plt.scatter(merge_df['Date'], merge_df['Enterococci (cfu/100ml)'])
#plt.semilogy(merge_df['Date'], merge_df['Enterococci (cfu/100ml)'])


# Set the x-axis label
plt.xlabel('Date')

# Set the y-axis label
plt.ylabel('Enterococci (cfu/100ml)')
#yscale("log")


# Show the plot
plt.show()


In [ ]:

# Plot the overall trend of Enterococci through time
import matplotlib.pyplot as plt

# Group the data by date and calculate the mean Enterococci value for each date
enterococci_by_date = merge_df.groupby('Date')['Enterococci (cfu/100ml)'].median()

# Plot the mean Enterococci values over time
plt.plot(enterococci_by_date)
plt.xlabel('Date')
plt.ylabel('Median Enterococci (cfu/100ml)')
plt.title('Overall Trend of Enterococci Through Time')
plt.show()
 

In [ ]:

# Rank and plot the sites by the amount of Enterococci through time

# Group data by Site and Date
df_grouped = df.groupby(['Site', 'Date'])['Enterococci (cfu/100ml)'].mean().reset_index().fillna(0)

In [ ]:
df_grouped

In [ ]:

# Create a pivot table to rank sites by Enterococci
df_pivot = df_grouped.pivot(index='Site', columns='Date', values='Enterococci (cfu/100ml)')


In [ ]:
df_pivot.fillna(0, inplace=True)

In [ ]:
df_pivot.mean(axis=1)

In [ ]:
df_pivot


In [ ]:
########

# TODO: Need to drop beaches not in the 160

In [ ]:
df_pivot.plot()

In [ ]:

# Sort the pivot table by highest average Enterococci
df_sorted = df_pivot.sort_values(by=df_pivot.mean(axis=1), ascending=False)
